In [0]:
import re
from nltk.stem import PorterStemmer    
ps = PorterStemmer() 

class features():
    def shape(self,word):    
        word_shape = 'other'
        if re.match('[0-9]+(\.[0-9]*)?|[0-9]*\.[0-9]+$', word):
            word_shape = 'number'
        elif re.match('\W+$', word):
            word_shape = 'punct'
        elif re.match('[A-Z][a-z]+$', word):
            word_shape = 'capitalized'
        elif re.match('[A-Z]+$', word):
            word_shape = 'uppercase'
        elif re.match('[a-z]+$', word):
            word_shape = 'lowercase'
        elif re.match('[A-Z][a-z]+[A-Z][a-z]+[A-Za-z]*$', word):
            word_shape = 'camelcase'
        elif re.match('[A-Za-z]+$', word):
            word_shape = 'mixedcase'
        elif re.match('__.+__$', word):
            word_shape = 'wildcard'
        elif re.match('[A-Za-z0-9]+\.$', word):
            word_shape = 'ending-dot'
        elif re.match('[A-Za-z0-9]+\.[A-Za-z0-9\.]+\.$', word):
            word_shape = 'abbreviation'
        elif re.match('[A-Za-z0-9]+\-[A-Za-z0-9\-]+.*$', word):
            word_shape = 'contains-hyphen'
        return word_shape

    def word2features(self,sent, i):
        word = sent[i][0]
        postag = sent[i][1]

        features = {
            'bias': 1.0,
            'word.lower()': word.lower(),
            'word[-3:]': word[-3:],
            'word[-2:]': word[-2:],
            'word.isupper()': word.isupper(),
            'word.istitle()': word.istitle(),
            'word.isdigit()': word.isdigit(),
            'postag': postag,
            'distfromsentbegin': i,
            'wordinitialcap': word[0].isupper(),
            'word.len': len(word),
            'word.shape': self.shape(word)
        }
        if i > 0:
            word1 = sent[i-1][0]
            postag1 = sent[i-1][1]
            features.update({
                '-1:word.stem': ps.stem(word1),
                '-1:word.len': len(word1), 
                '-1:word[-3:]': word1[-3:],
                '-1:word[-2:]': word1[-2:],
                '-1:word.lower()': word1.lower(),
                '-1:word.istitle()': word1.istitle(),
                '-1:word.isupper()': word1.isupper(),
                '-1:postag': postag1,
                '-1:postag': self.shape(word1)
            })
        else:
            features['BOS'] = True

        if i < len(sent)-1:
            word1 = sent[i+1][0]
            postag1 = sent[i+1][1]
            features.update({
                '+1:word.stem': ps.stem(word1),
                '+1:word.len': len(word1),
                '+1:word[-3:]': word1[-3:],
                '+1:word[-2:]': word1[-2:],
                '+1:word.lower()': word1.lower(),
                '+1:word.istitle()': word1.istitle(),
                '+1:word.isupper()': word1.isupper(),
                '+1:postag': postag1,
                '+1:postag': self.shape(word1)
            })
        else:
            features['EOS'] = True

        return features

    def sent2features(self,sent):
        return [self.word2features(sent, i) for i in range(len(sent))]

    def sent2labels(self,sent):
        return [label for token, postag, label in sent]

    def sent2tokens(sent):
        return [token for token, postag, label in sent]